<a href="https://colab.research.google.com/github/TranPhu1999/YOLO3-face-mask-detection/blob/main/train_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- IMPORT THƯ VIỆN

In [ ]:
from IPython.display import Image, display
import xml.etree.ElementTree as ET
import os
from sklearn.model_selection import train_test_split
import shutil # High-level file operations
import tqdm # show progress bar
from datetime import datetime # để đo thời gian train
# import utils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/A_ Face detection'

/content/drive/MyDrive/A_ Face detection


- TẢI YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15874, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 15874 (delta 46), reused 45 (delta 15), pack-reused 15769
Receiving objects: 100% (15874/15874), 14.63 MiB | 10.77 MiB/s, done.
Resolving deltas: 100% (10860/10860), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.9/606.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Tải các file trọng số pretrained weights *.pt
!cd /content/yolov5/models/hub && wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt

/bin/bash: line 1: cd: /content/yolov5/models/hub: No such file or directory


# Chuẩn bị dữ liệu

In [ ]:
import os

len(os.listdir('/content/drive/MyDrive/A_ Face detection/dataset/images'))

972

In [ ]:
%cd '/content/drive/MyDrive/A_ Face detection'

/content/drive/MyDrive/A_ Face detection


- TRAIN TEST SLIPT

In [ ]:
import os
from sklearn.model_selection import train_test_split

In [ ]:
all_files = os.listdir('/content/drive/MyDrive/A_ Face detection/dataset/images')
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=1337)
train_files, val_files =  train_test_split(train_files, test_size=0.2, random_state=1337)

print('Train files:', len(train_files))
print('Val files:', len(val_files))
print('Test files:', len(test_files))

Train files: 640
Val files: 160
Test files: 200


# Tạo thư mục dataset theo cấu trúc thư mục dataset của YOLO

In [ ]:
Thư mục dataset (đặt tên là face) chứ các folder sau:
Folder train: gồm 2 folder images & labels:
Folder val: tương tự
Folder test: tương tự

In [ ]:
import os

current_directory = os.getcwd()
print("Bạn đang đứng ở thư mục:", current_directory)

Bạn đang đứng ở thư mục: /content


In [ ]:
%cd '/content/drive/MyDrive/A_ Face detection'

/content/drive/MyDrive/A_ Face detection


In [ ]:
! mkdir ./face/ # tạo folder dataset tên 'face'

In [ ]:
# folder 'train'
! mkdir -p ./face/train/images
! mkdir -p ./face/train/labels
# folder 'val'
! mkdir -p ./face/val/images
! mkdir -p ./face/val/labels
#folder 'test'
! mkdir -p ./face/test/images
! mkdir -p ./face/test/labels


In [ ]:
%cd '/content/drive/MyDrive/A_ Face detection/dataset'

/content/drive/MyDrive/A_ Face detection/dataset


- ĐƯA CÁC FILE VÀO THƯ MỤC TƯƠNG ỨNG

In [ ]:
img_base = '/content/drive/MyDrive/A_ Face detection/dataset/images'
yolo_base = '/content/drive/MyDrive/A_ Face detection/dataset/list_bbox_celeba.txt'

def move_files(files, type='train'):
  for file in tqdm.tqdm(files): # tqdm: show progess bar
    img_src = os.path.join(img_base, file)
    ann_src = os.path.join(yolo_base, file.split('.')[0] + '.txt')
    img_dst = './face/' + type +'/images/' + file
    ann_dst = './face/' + type +'/labels/' + file.split('.')[0] + '.txt'
    shutil.copy(img_src, img_dst)
    shutil.copy(ann_src, ann_dst)
  print('Complete move', type, 'files')

move_files(train_files, 'train')
move_files(test_files, 'test')
move_files(val_files, 'val')

  0%|          | 0/640 [00:00<?, ?it/s]


FileNotFoundError: ignored

# 3. TRAINING AND TEST

# 3.1 Training from scratch Dưới đây là mô tả các tham số cần thiết để huấn luyện mô hình YOLOv5 và ý nghĩa của chúng:

3.1 Training from scratch
Dưới đây là mô tả các tham số cần thiết để huấn luyện mô hình YOLOv5 và ý nghĩa của chúng:

'batch': Kích thước batch dùng trong quá trình huấn luyện. Đặt -1 để sử dụng batch size tối đa mà phần cứng của bạn hỗ trợ.

'epochs': Số lượng epoch, tức là số lần huấn luyện sẽ thực hiện trên toàn bộ tập dữ liệu. Đặt giá trị nhỏ hơn để huấn luyện nhanh hơn.

'data': Đường dẫn đến tệp cấu hình dữ liệu (data-configurations file). Tệp cấu hình dữ liệu chứa thông tin về tập dữ liệu, bao gồm đường dẫn đến các tệp dữ liệu và số lượng lớp.

'cfg': Đường dẫn đến tệp cấu hình mô hình (model-configurations file). Tệp cấu hình mô hình chứa thông tin về kiến trúc của mô hình YOLOv5 được sử dụng.

'weights': Đường dẫn đến trọng số ban đầu (initial weights). Bạn có thể sử dụng các trọng số đã được huấn luyện từ các mô hình có sẵn (pretrained models). Kiến trúc mô hình sẽ phụ thuộc vào tệp trọng số được cung cấp.

'cache': Lưu ảnh vào bộ nhớ cache để tăng tốc độ huấn luyện.

'img': Kích thước ảnh dùng trong quá trình huấn luyện. Mặc định là 640 pixels.

In [ ]:
start = datetime.now()
!python yolov5/train.py --batch 32 --epochs 100 --data ./facemask/facemask.yaml \
                        --weights '' --cfg '/content/drive/MyDrive/A_ Face detection/yolov5/models/hub/yolov5s6.yaml' \
                        --patience 50
end = datetime.now()
print(f'training time: {end-start}')

# 3.2. Training using pretrained weights
- Bây giờ ta sẽ train model sdung pretrained weights từ mô hình yolov5s6. Warm up là quá trình cần thiết để mô hình hội tụ nhanh hơn. Warm up sẽ đóng băng lại các layers CNN để cho hệ số của chúng không đổi và chỉ train lại trên các Fully Conntected Layers ở cuối cùng. Mục đích của warm up là giữ nguyên được các đặc trưng bậc cao (high-level) đã được học từ pretrained-model
- Mặt khác, model thường gồm 2 component: backbone layers (đóng vai trò như feature extractor) & head layers (tính toán gtri output cho prediction). Trong code cell này, ta thử sử dụng cùng backbone layers giống với yolov5s6 (i.e. giữ nguyên 12 layers), và chỉ train phần model's head layers. Như vậy ta freeze 12 layers đầu
- Tham số ta truyền cũng giống trên, bsung thêm:
freeze — number of layers to freeze
project — name of the project
name — name of the run
- Nếu 2 tham số project='runs_facemask' & name='feature_extraction' dc specify thì kqua sẽ dc lưu tại folder /content/runs_facemask/feature_extraction/, còn ko thì mặc định lưu tại folder yolov5/runs/train

In [ ]:
start = datetime.now()
!python yolov5/train.py --batch 32 --epochs 100 --data ./facemask/facemask.yaml \
                        --weights yolov5/models/hub/yolov5s6.pt --cache --patience 50 \
                        --freeze 12 --project 'runs_facemask' --name 'feature_extraction' --exist-ok
end = datetime.now()
print(f'training time: {end-start}')

# ĐÁNH GIÁ THUẬT TOÁN BẰNG MA TRẬN NHẦM LẪN

# NHẬN DIỆN GƯƠNG MẶT BẰNG CAMERA